# Testing
### Eintragung und Auflösung von Partnerschaften nach Geschlecht, seit 2014
Datum: 02.11.2021

**Importiere die notwendigen Packages**

In [1]:
#pip install geopandas fiona requests folium mplleaflet contextily

In [18]:
import pandas as pd
import pivottablejs
from pivottablejs import pivot_ui
import numpy as np
import altair as alt
import matplotlib.pyplot as plt
from datetime import datetime
import geopandas as gpd
import folium 



Definiere Settings. Hier das Zahlenformat von Float-Werten (z.B. *'{:,.2f}'.format* mit Komma als Tausenderzeichen), 

In [19]:
pd.options.display.float_format = '{:.0f}'.format
pd.set_option('display.width', 100)
pd.set_option('display.max_columns', 15)

Importiere einen Datensatz von der **DWH_INT** Dropzone.

- Beachte dabei die Notation des Pfades...
- Definiere mal aktuell noch keine weiteren Parameter beim Import

**Dataset auf INTEG Datenkatalog**: https://data.integ.stadt-zuerich.ch/dataset/int_dwh_bev_partnerschaft_geschlecht_seit2014_od3320

(int_dwh)

In [20]:
# Filepath
fp = r"\\szh\ssz\applikationen\OGD_Dropzone\INT_DWH\bev_partnerschaft_geschlecht_seit2014_od3320\BEV332OD3320.csv"


# Read the data
data2betested = pd.read_csv(
    fp, 
    sep=',',
   #usecols=[0,1,2,3,4,6],
   #names=['stationid','year','month','avgtemp','latitude','longitude','elevation','station','countryid','country'],
   #skiprows=1,
   # parse_dates=[0],
    parse_dates=True,
    #date_parser=lambda x: pd.to_datetime(x, format="%Y"), 
    #index_col="date_time",
    low_memory=False    
    )


In der Folge ein paar erste Tests:

In [21]:
type(data2betested)

pandas.core.frame.DataFrame

 - 1) Zeige eine kurze Vorschau der importierten Daten
 - 2) Weise die Datentypen aus
 - 3) Zeige die Shape (Umfang) des Datensatzes an

In [22]:
data2betested.head(3)

,EreignisDatJahr,SexLang,SexCd,Partnerschaft,PartnerschaftSort,Anzahl
0,2014,weiblich,2,Eingetragene Partnerschaft,1,30
1,2014,männlich,1,Eingetragene Partnerschaft,1,85
2,2014,weiblich,2,Aufgelöste Partnerschaft,2,6


In [23]:
data2betested.dtypes

EreignisDatJahr       int64
SexLang              object
SexCd                 int64
Partnerschaft        object
PartnerschaftSort     int64
Anzahl                int64
dtype: object

In [24]:
data2betested.shape

(28, 6)

Beschreibe einzelne Attribute

In [25]:
data2betested.describe()

,EreignisDatJahr,SexCd,PartnerschaftSort,Anzahl
count,28,28,28,28
mean,2017,2,2,40
std,2,1,1,35
min,2014,1,1,6
25%,2015,1,1,16
50%,2017,2,2,29
75%,2019,2,2,46
max,2020,2,2,122


Wie viele Nullwerte gibt es im Datensatz?

In [26]:
data2betested.isnull().sum()

EreignisDatJahr      0
SexLang              0
SexCd                0
Partnerschaft        0
PartnerschaftSort    0
Anzahl               0
dtype: int64

Wandle die Spalte Jahr von Int zu **Datetime** um. Dies wird in den meisten Grafiken so benötigt.

In [28]:

data2betested['StichtagDatJahr'] = data2betested['EreignisDatJahr']

data2betested['StichtagDatJahr']=pd.to_datetime(data2betested['StichtagDatJahr'], format='%Y')

#TO DO --> setze das Datum auf den 31.12. der Jahres.
#test=data2betested[['StichtagDatJahr']].date().replace(month=12, day=31)

data2betested

,EreignisDatJahr,SexLang,SexCd,Partnerschaft,PartnerschaftSort,Anzahl,StichtagDatJahr
0,2014,weiblich,2,Eingetragene Partnerschaft,1,30,2014-01-01
1,2014,männlich,1,Eingetragene Partnerschaft,1,85,2014-01-01
2,2014,weiblich,2,Aufgelöste Partnerschaft,2,6,2014-01-01
3,2014,männlich,1,Aufgelöste Partnerschaft,2,23,2014-01-01
4,2015,weiblich,2,Eingetragene Partnerschaft,1,26,2015-01-01
5,2015,männlich,1,Eingetragene Partnerschaft,1,100,2015-01-01
6,2015,weiblich,2,Aufgelöste Partnerschaft,2,9,2015-01-01
7,2015,männlich,1,Aufgelöste Partnerschaft,2,31,2015-01-01
8,2016,weiblich,2,Eingetragene Partnerschaft,1,26,2016-01-01
9,2016,männlich,1,Eingetragene Partnerschaft,1,122,2016-01-01


Bereite die Daten für die Grafik unten vor

In [29]:
Jahre = data2betested.groupby("StichtagDatJahr")
len(Jahre)

7

In [30]:
for key, group in Jahre:
    print(key, len(group))

2014-01-01 00:00:00 4
2015-01-01 00:00:00 4
2016-01-01 00:00:00 4
2017-01-01 00:00:00 4
2018-01-01 00:00:00 4
2019-01-01 00:00:00 4
2020-01-01 00:00:00 4


In [31]:
myJahr = "2020-01-01"
aktuellesJahr=Jahre.get_group(myJahr)   
aktuellesJahr

,EreignisDatJahr,SexLang,SexCd,Partnerschaft,PartnerschaftSort,Anzahl,StichtagDatJahr
24,2020,weiblich,2,Eingetragene Partnerschaft,1,27,2020-01-01
25,2020,männlich,1,Eingetragene Partnerschaft,1,83,2020-01-01
26,2020,weiblich,2,Aufgelöste Partnerschaft,2,12,2020-01-01
27,2020,männlich,1,Aufgelöste Partnerschaft,2,28,2020-01-01


**Selektionen und Aggregationen**

Check this [Doku](https://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/)

In [32]:
#test = data2betested.groupby(['StichtagDatJahr', 'AlterV20Kurz'])[['AnzBestWir']].agg()

# produces Pandas Series
    #data2betested.groupby(['StichtagDatJahr', 'KonOGDSort'])['AnzBestWir'].sum()
# Produces Pandas DataFrame
    #data2betested.groupby(['StichtagDatJahr', 'KonOGDSort'])[['AnzBestWir']].sum()
    

**Einfache Visualisierungen zur Plausi**

Exploriere die Daten mit Pivottable.JS

In [33]:
from pivottablejs import pivot_ui

pivot_ui(data2betested)

Liniendiagramm 
[Link zur Doku](https://altair-viz.github.io/gallery/multiline_highlight.html)

In [34]:
data2betested.head(2)

,EreignisDatJahr,SexLang,SexCd,Partnerschaft,PartnerschaftSort,Anzahl,StichtagDatJahr
0,2014,weiblich,2,Eingetragene Partnerschaft,1,30,2014-01-01
1,2014,männlich,1,Eingetragene Partnerschaft,1,85,2014-01-01


In [65]:
# df_statzonen = df_statzonen[(df_statzonen.knr>0) & (df_statzonen.qname!="Hirslanden")]
mySelection = data2betested[(data2betested.PartnerschaftSort == 1) & (data2betested.SexLang == "männlich")]
mySelection2 = data2betested[(data2betested.PartnerschaftSort == 2) & (data2betested.SexLang == "männlich")]
mySelection3 = data2betested[(data2betested.PartnerschaftSort == 1) & (data2betested.SexLang == "weiblich")]
mySelection4 = data2betested[(data2betested.PartnerschaftSort == 2) & (data2betested.SexLang == "weiblich")]

highlight = alt.selection(type='single', on='mouseover',
                          fields=['Partnerschaft', 'SexLang'], nearest=True)
#x='date:StichtagDatJahr',
base = alt.Chart(mySelection).encode(
    x='StichtagDatJahr',
    y='Anzahl',
    color='Partnerschaft'
)

second = alt.Chart(mySelection2).encode(
    x='StichtagDatJahr',
    y='Anzahl',
    color='Partnerschaft'
)

third = alt.Chart(mySelection3).encode(
    x='StichtagDatJahr',
    y='Anzahl',
    color='Partnerschaft'
)

fourth = alt.Chart(mySelection4).encode(
    x='StichtagDatJahr',
    y='Anzahl',
    color='Partnerschaft'
)


points = base.mark_circle().encode(
    opacity=alt.value(2)
).add_selection(
    highlight
).properties(
    width=700 , height=300
)

lines4 = fourth.mark_line().encode(
    size=alt.condition(~highlight, alt.value(1.5), alt.value(4))
)

lines3 = third.mark_line().encode(
    size=alt.condition(~highlight, alt.value(1.5), alt.value(4))
)

lines2 = second.mark_line().encode(
    size=alt.condition(~highlight, alt.value(1.5), alt.value(4))
)

lines = base.mark_line().encode(
    size=alt.condition(~highlight, alt.value(1.5), alt.value(4))
).interactive()



points + lines + lines3 

alt.LayerChart(...)

In [56]:
alt.Chart(data2betested).mark_circle(size=60).encode(
    x='StichtagDatJahr',
    y='Anzahl',
    #color='AnzNat',
    #color=alt.Color('AnzNat', scale=alt.Scale(scheme='greens')),
    color=alt.Color('Partnerschaft', scale=alt.Scale(scheme='dark2')),
    tooltip=['StichtagDatJahr','Partnerschaft','Anzahl']
).interactive() # this makes the axes interactive: now you can zoom & pan




alt.Chart(...)

**Sharepoint als gecheckt markieren!**

Record auf Sharepoint: **[Link](http://kollaboration.intranet.stzh.ch/orga/ssz-produkte/Lists/SASA_Outputs/EditForm.aspx?ID=52&Source=%2Forga%2Fssz%2Dprodukte%2FLists%2FSASA%5FOutputs)**